<a href="https://colab.research.google.com/github/sadnow/AnimationKit-AI/blob/testing/Dev_SadKit's_AnimationKit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sadnow's AnimationKit - Synthetic Video Tools

**ALPHA 2.8: Warning - Testing branch; This rev has only been tested with default settings.**

##Current Changelog:
- Completely tossed code, starting over

##2.6 Changelog:

- Partial implementation of method-b for imported_mp4s
- Succesful implementation and testing of RIFE animevideo

Not completely implemented/tested:

- audio support
- method-a (previous method) untested
- other models untested
- indiviudal frame support (untested)


---

Main (stable) branch and any potential updates can be found here: https://github.com/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN

---
#Credits

Motion smoothing utilizes [Practical-RIFE](https://github.com/hzwer/Practical-RIFE) 4.0 model.

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833). A demo can be found at https://raw.githubusercontent.com/xinntao/Real-ESRGAN

In [ ]:
# @title AnimationKit AI License(s): { form-width: "300px" }
# @markdown Practical-RIFE uses the MIT License.

# @markdown Real-ESRGAN uses the BSD 3-Clause License.

# @markdown *To view the included licenses, click "Show code."* 
# -------------------------------------------------
# Below is the MIT license for RIFE & Practical-RIFE
# ------------------------------------------------

# MIT License

# Copyright (c) 2021 hzwer

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# -------------------------------------------------
# Below is the BSD3-Clause License for Real-ESRGAN.
# ------------------------------------------------

# BSD 3-Clause License

# Copyright (c) 2021, Xintao Wang
# All rights reserved.

# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:

# 1. Redistributions of source code must retain the above copyright notice, this
#    list of conditions and the following disclaimer.

# 2. Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.

# 3. Neither the name of the copyright holder nor the names of its
#    contributors may be used to endorse or promote products derived from
#    this software without specific prior written permission.

# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.




# *Sadnow's AnimationKit AiO*


In [ ]:
#@title Check GPU, Load Libraries, Install Dependencies { form-width: "300px" }
#@markdown
#@markdown ***This Notebook supports "Run All. Once everything is set to your liking, you can do `Ctrl+F9`!***
import torch
# Check the GPU status
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

enable_error_checking = True #Setting to 'False' saves a lil ram
if enable_error_checking:
  !nvidia-smi
else:
  !nvidia-smi
  !nvidia-smi -i 0 -e 0
#@markdown *Special thanks to sportracer48 and his Discord channel. If you want to make AI animations, PyTTI is king.* https://www.patreon.com/sportsracer48

#@markdown ---
_preserve_in_gdrive = True #@param {type:"boolean"}
#@markdown > This toggle saves the current version of AnKit and some of its dependencies to `drive/AI/AnimationKit` and loads those on startup.

################################
def Install_ESRGAN():
  print("Installing libraries for Real-ESRGAN upscaling.")
  #!git clone https://github.com/xinntao/Real-ESRGAN.git &> /dev/null #using own fork for longevity
  !git clone https://github.com/sadnow/Real-ESRGAN_AnimationKit Real-ESRGAN &> /dev/null #using own fork for longevity
  os.chdir('/content/Real-ESRGAN')
  !pip -q install basicsr
  !pip -q install facexlib
  !pip -q install gfpgan
  !pip -q install -r requirements.txt   &> /dev/null
  !python setup.py develop              &> /dev/null
  if _upscale_model_name == 'RealESRGAN_x4plus':
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models              &> /dev/null
  if _upscale_model_name == 'RealESRGAN_x4plus_anime_6B':  
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models   &> /dev/null
  if _upscale_model_name == 'RealESRGAN_x2plus':
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models              &> /dev/null
  if _upscale_model_name == 'RealESRGANv2-animevideo-xsx2':
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.3.0/RealESRGANv2-animevideo-xsx2.pth -P experiments/pretrained_models
  !pip -q install ffmpeg-python
def Install_RIFE():
  os.chdir('/content/')
  print("Installing libraries for RIFE motion smoothing.")
  !git clone https://github.com/sadnow/Practical-RIFE_AnimationKit Practical-RIFE &> /dev/null  #using my own archive for longevity
  !gdown --id 1mUK9iON6Es14oK46-cCflRoPTeGiI_A9 &> /dev/null  #NEWER 4.0 MODEL
  !7z e RIFE_trained_model_v4.0.zip &> /dev/null
  !mkdir /content/Practical-RIFE/train_log  &> /dev/null
  !mv *.py /content/Practical-RIFE/train_log/ &> /dev/null
  !mv *.pkl /content/Practical-RIFE/train_log/  &> /dev/null
  os.chdir('/content/Practical-RIFE/')
  !pip3 install -r requirements.txt &> /dev/null
  print("Finsihed Installing libraries for RIFE motion smoothing.")

#################################
class SadKit:
  def __init__(self):
    self.pathtool = self.Pathtool()
    self.image_tools = self.Image_tools()
    self.video = self.Video()
    import os.path
    from os import path
  def dprint(self,input_str): #debug print
    #3 types of logs: info, warning, error
    #info - for dev tracking; warning - for when program has to override user settings
    #print input_str in red, highlighted text:
    global dprint_log
    print("\033[1;31;40m",input_str,"\033[0m")
    dprint_log = dprint_log + input_str + "\n"
    return dprint_log
  class Image_tools:
    def images2video(self, input):
      #!ffmpeg -framerate $_target_fps -pattern_type glob -i '*.png' -y '/content/frames2video.mp4' #old method
      print("RUNNING THE EXPERIMENTAL")
      !ffmpeg -r 24 -pattern_type glob -i '*.png' -c:v libx264 -r 23.98 -pix_fmt yuv420p '/content/frames_storage/init_frame_storage/frames2video.mp4'  #documented example realsrgan-video.md
        #note: detect_fps might be wonky
  class Video:
    def upscale(self, input_file, output_dir, model, params):
      os.chdir('/content/Real-ESRGAN')
      !python inference_realesrgan_video.py --input $input_file --output $output_dir --model_name $model --suffix realesrgan --video $params
  class Pathtool:
    #intent(s): detect, create, insert, check_type, remove_extension, append_prefix, append_suffix
    def drive_mount(self):
      from google.colab import drive
      drive.mount('/content/drive')
    def is_valid_filename(self, file_input):  #identify whether path/string has valid namescheme (even if nonexistent)
      file_input = os.path.basename(file_input)
      #print(len(dir_input)) #count total # of characters
      if file_input.count('.') == 1: return True
      elif file_input.count('.') > 1:
        dprint("ERROR: This currently can't process multiple-dotted filenames.")
        return False
      else: return False
    def mkdir(self, input):
      pathlib.Path(input).mkdir(parents=True, exist_ok=True)
      #os.mkdir(input)
    def check_type(self, input): #determine whether file or directory
      #types: file,dir
      if os.path.isdir(input):  type = 'dir'
      elif os.path.isfile(input): type = 'file'
      else: type = 'error'
      #dprint("'Type of input is ',type")
      return type
    def sanitize(self, input):
      if self.is_valid_filename(input):
        input = os.path.normpath(input)
        return input
      else: 
        dprint("Faliure to sanitize input: Invalid filename.")
        exit()
    def dir_copy_by_extension(self, input_dir, output_dir, file_ext): #sorts a dir's files by extension and copies matching files to new dir
      #print("\n Beginning sortFrames... ")
      if not self.isfileordir(output_dir):
        self.mkdir(output_dir)
      #print("\n Copying frames to "+'/content/frames_storage/init_frame_storage'+" for processing...\n")
      os.chdir(input)
      #!find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t '/content/frames_storage/init_frame_storage'
      !find -maxdepth 1 -name '*.'$file_ext -print0 | xargs -0 cp -t $output
      os.chdir(output)
      !find . -type f -name "*."$file_ext -execdir bash -c 'mv "$0" "${0##*_}"' {} \;  #removes anything not numbered (junk files) from new dir
      self.pad_file_suffix_bulk(output_dir)
      print("\n Finished copying frames to "+'/content/frames_storage/init_frame_storage'+".\n")
      print("Completed sortFrames... \n")
    def pad_file_suffix_bulk(input_dir): #pads all filenames within a dir
      os.chdir(input_dir)
      !rename 's/\d+/sprintf("%08d",$&)/e' *  #adds padding to numbers
      return
    def isfileordir(self, input):
      if self.check_type(input) == 'file' or self.check_type(input) == 'dir': return True
      else: return False
    def omit_extension(self, input_file):
      next = (os.path.splitext(input_file)[0])
      return next
    def upload_(self, input, output):
      from google.colab import files
      import shutil
      upload_folder = input
      result_folder = output
      if os.path.isdir(upload_folder):
        shutil.rmtree(upload_folder)
      if os.path.isdir(result_folder):
        shutil.rmtree(result_folder)
      os.mkdir(upload_folder)
      os.mkdir(result_folder)
      uploaded = files.upload()
      for filename in uploaded.keys():
        dst_path = os.path.join(upload_folder, filename)
        print(f'move {filename} to {dst_path}')
        shutil.move(filename, dst_path)
    def determine_filesuffix(input_file):
      #determine the suffix of a file
      suffix = os.path.splitext(input_file)[1]
      return suffix
    def file_addsuffix(input,suffix): #identify the potential output name of an upscaled file
      input = Pathtool.omit_extension(input)
      input = input + '_' + suffix
      return input
    def overwrite_protect(input): #adds randomized suffix to filename
      from datetime import datetime
      current_time = datetime.now().strftime('%y%m%d-%H%M%S_%f')
      filesuffix = self.determine_filesuffix(input)
      input = self.omit_extension(input)
      input = input + '_' + current_time + filesuffix
      input = Pathtool.sanitize(input)
      dprint("NOTICE: Overwrite protection has renamed your output file to " + input)
      return input
#---end of classwork--------------------
#############################################################
Sadkit = SadKit()
Pathtool = Sadkit.Pathtool()
Image_tools = Sadkit.Image_tools()
Video = Sadkit.Video()
dprint = Sadkit.dprint

def install_dependencies():
  if _enable_upscaling:
    if not os.path.isdir('/content/Real-ESRGAN'):
      Install_ESRGAN()
    #print("Finished Installing libraries for Real-ESRGAN upscaling.")
  if _enable_motion_smoothing:
    if not os.path.isdir('/content/Practical-RIFE'):
      Install_RIFE()
    #print("Finished Installing libraries for RIFE motion smoothing.")

In [ ]:
import os
import pathlib
#@title Sadnow's AnimationKit { form-width: "300px" }
#@markdown This cell combines Real-ESRGAN upscaling, RIFE motion smoothing/interpolation, and everything else.

#@markdown ---
_input_path = '/content/drive/MyDrive/VQLIPSE/videos/filespace-dinner.mp4' #@param{type:"string"}
#@markdown >*`_input_path` can be an mp4 file or a frame folder (containing PNG's). Ignore this setting if not using Google Drive.*
_use_google_drive = True #@param {type:"boolean"}
#@markdown > Google Drive is optional. Non-gDrive users will be prompted with an "Upload" box.
_output_path = '/content/drive/MyDrive/AI/AnimationKit/videos/dinner_AnKit.mp4' #@param {type:"string"}
_keep_source_audio = False #@param{type:"boolean"}
#@markdown ---
_enable_upscaling = True #@param{type:"boolean"}
_upscale_model_name='RealESRGANv2-animevideo-xsx2' #@param ['RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus','RealESRGANv2-animevideo-xsx2','RealESRGANv2-animevideo-xsx4'] {type:"string"}
_upscaling_face_enhance=True #@param{type:"boolean"}
_realesrgan_half_precision=True #@param{type:"boolean"}
#@markdown ---
_enable_motion_smoothing = True #@param{type:"boolean"}
_output_target_length_seconds=30#@param {type:"integer"}
_output_target_fps=60#@param {type:"integer"}
_RIFE_scale='1.0'#@param ['0.25','0.5','1.0','2.0']{type:"string"}
#@markdown ---
_output_encoder='h265' #@param ['lossless','h264','h265'] {type:"string"}
_output_quality=32#@param {type:"slider", min:20, max:50, step:1}
#@markdown > *Default `_constant_quality` is `30`. Higher values = lower filesize, lower quality.*
_debug_mode = True #@param{type:"boolean"}

#input_path='/content/drive/MyDrive/pytti_test/videos/zelda4.mp4' #@param {type:"string"}

##############################################################
####################  PREPARATION  ###########################
if _use_google_drive:
  Pathtool.drive_mount()
dprint_log = ""
###debug#

###

install_dependencies()
os.chdir('/content/')

###path handling###
  #todo: garbage collection
tmp_source_frames = '/content/user_files/source_frames'
tmp_upscaled_frames = '/content/user_files/upscaled_frames'
if not _use_google_drive: Pathtool.upload_(tmp_source_frames) #gdrive alternative
Pathtool.mkdir(tmp_source_frames)
Pathtool.mkdir(tmp_upscaled_frames)
_input_path = Pathtool.sanitize(_input_path)
input_type = Pathtool.check_type(_input_path)
if input_type == 'file':  input_filename = os.path.basename(_input_path)
input_dir = os.path.dirname(_input_path)
_output_path = Pathtool.sanitize(_output_path)
output_type = Pathtool.check_type(_output_path)
if output_type == 'file': output_filename = os.path.basename(_output_path)
output_dir = os.path.dirname(_output_path)
Pathtool.mkdir(output_dir)
if not Pathtool.isfileordir(_input_path): dprint("ERROR! Check your _input_path to see if it is a valid file or directory.")
if Pathtool.isfileordir(_output_path): dprint("ERROR! Check your _output_path to see if it is a valid file or directory.")

###real-esr PARAM handling###
realesrgan_params = ''
def add_argument(input_string,output):
  output = output + input_string + ' '
  return output
if _upscaling_face_enhance: realesrgan_params = add_argument('--face_enhance', realesrgan_params)
if _realesrgan_half_precision:
  realesrgan_params = add_argument('--half', realesrgan_params)
  dprint("INFO: _realesrgan_half_precision might yield negligible performance improvements depending on the GPU.")
if _keep_source_audio:  realesrgan_params = add_argument('--audio', realesrgan_params)
# add_argument('--outscale', type=float, default=4, help='The final upsampling scale of the image')
# add_argument('--tile', type=int, default=0, help='Tile size, 0 for no tile during testing')
# add_argument('--tile_pad', type=int, default=10, help='Tile padding')
# add_argument('--pre_pad', type=int, default=0, help='Pre padding size at each border')
# add_argument('--face_enhance', action='store_true', help='Use GFPGAN to enhance face')
# add_argument('--video', action='store_true', help='Output a video using ffmpeg')
# add_argument('--fps', type=float, default=None, help='FPS of the output video')
# add_argument('--consumer', type=int, default=4, help='Number of IO consumers')

###################################################################
#####################  UPSCALING  #################################
#non-mp4 pre-upscaling#(untested)
if input_type == 'dir':
  Pathtool.copy_by_extension(input_dir,tmp_source_frames)
  Image_tools.images2video(input_dir,_output_path)

###mp4 upscaling
  #def upscale(self, input_file, output_dir, model, params):
Video.upscale(_input_path,tmp_upscaled_frames,_upscale_model_name,realesrgan_params)

upscaled_path = Pathtool.omit_extension(_input_path)
upscale_opath = file_addsuffix()  #this is the name of the nonexistent upscaled file
if Pathtool.isfileordir(id_upscaled_path()): print("NOTICE: Upscale Succesful!")
else: print("FAIL!")
###

#SadKit().Path().check_type(_input_path)


In [ ]:
#@title Testing Sandbox { form-width: "300px" }
#@markdown This is for testing different functions. *Do* **not** delete this cell!
_use_google_drive = True #@param {type:"boolean"}
_input_path = '/content/drive/MyDrive/VQLIPSE/videos/filespace-dinner.mp4' #@param{type:"string"}
_use_google_drive = True #@param {type:"boolean"}
_output_path = '/content/drive/MyDrive/AI/AnimationKit/videos/dinner_AnKit.mp4' #@param {type:"string"}
_keep_source_audio = False #@param{type:"boolean"}
_enable_upscaling = True #@param{type:"boolean"}
_upscale_model_name='RealESRGANv2-animevideo-xsx2' #@param ['RealESRGAN_x4plus','RealESRGAN_x4plus_anime_6B','RealESRGAN_x2plus','RealESRGANv2-animevideo-xsx2','RealESRGANv2-animevideo-xsx4'] {type:"string"}
_upscaling_face_enhance=True #@param{type:"boolean"}
_realesrgan_half_precision=True #@param{type:"boolean"}
_enable_motion_smoothing = True #@param{type:"boolean"}
_output_target_length_seconds=30#@param {type:"integer"}
_output_target_fps=60#@param {type:"integer"}
_RIFE_scale='1.0'#@param ['0.25','0.5','1.0','2.0']{type:"string"}
_output_encoder='h265' #@param ['lossless','h264','h265'] {type:"string"}
_output_quality=32#@param {type:"slider", min:20, max:50, step:1}
_debug_mode = True #@param{type:"boolean"}

if _use_google_drive:
  Pathtool.drive_mount()
###########################################################
import os
import pathlib

# def file_findsuffix(input):
#   file_extension = pathlib.Path(input).suffix
#   print(file_findsuffix)
# def file_addsuffix(input,suffix):
#   print(input)
# # function to return the file extension
  
#   print("File Extension: ", file_extension)
#   #if not guess == '.mp4': print("ERROR: Extension needs to be mp4!")
#   input = Pathtool.omit_extension(input)
#   input = input + '_' + suffix
#   return input
# file = '/content/aids.mp4'
# #file_addsuffix(input=_input_path,suffix='_realesrgan.mp4'
# #print(file_findsuffix(_input_path))

# import pathlib
  
# # function to return the file extension
# file_extension = pathlib.Path('my_file.txt').suffix
# print("File Extension: ", file_extension)

#############new codex attempts

def determine_filesuffix(input_file):
    #determine the suffix of a file
    suffix = os.path.splitext(input_file)[1]
    return suffix
input_suffix = determine_filesuffix('test')
print(input_suffix)

This Notebook has been designed so you can "Run All". [**Ctrl+F9**]